\tableofcontents

\pagebreak

# Consideraciones Generales

Este informe documenta el desarrollo de una aplicación de línea de comandos para adquisición y gestión de datos desde un microcontrolador, utilizando C++17 y una arquitectura simple orientada a objetos. La herramienta fue probada con hardware real y contempla lectura/escritura de archivos, conversión entre formatos y comunicación serie.

# Esquema general de la solución

## Diseño de la solución
Se adoptó una arquitectura simple orientada a objetos, respetando el diagrama de clases provisto en la consigna. El diagrama actualizado se ilustra a continuación:


- **App** coordina la ejecución general y delega en `CLI` la interpretación de argumentos.
- **CLI** encapsula la lectura de los parámetros y expone *getters* tipados (`Mode`, `IOFormat`).
- **File** maneja la persistencia en CSV, mantiene los metadatos (`FileInfo`) y ofrece conversores a JSON/XML.
- **SerialPort** abstrae la comunicación por puerto serie usando `termios` para trabajar en modo crudo a 115200 bauds.
- **Types** concentra tipos comunes (`AppError`, `Mode`, `IOFormat`) y las funciones utilitarias `parseLine` y `tryParseHeaderTag`.

![alt text](DiagramUmbrello.png)


# Interfaces de usuario

La interfaz es puramente CLI. Los casos de uso principales son:

- Escritura (captura desde µC a CSV)
```bash
./app -m w -i c -n 20 -s /dev/ttyACM0 -d logs -f sensor
```

Lectura en distintos formatos

- CSV:       
    
```bash
./app -m r -o c -d logs -f sensor
```
 
- JSON:


```bash
./app -m r -o j -d logs -f sensor
```

- XML:

```bash
./app -m r -o x -d logs -f sensor
```

Luego, tenemos las siguientes capturas de pantalla:

![alt text](Imagenes/captura1.png)

![alt text](Imagenes/captura2.png)


Cabe resaltar que usar el comando `cat` me muestra los datos del archivo. Esto quiere decir que la **persistencia** se realiza en formato `.csv`.


![alt text](Imagenes/captura_csv.png)

![alt text](Imagenes/captura_json.png)

![alt text](Imagenes/captura_xml.png)


# Recursos adicionales

No se hace uso de ningun componente de software no estandar del lenguaje ni de la plataforma para codificar el programa. A continuación, se listan las librerias estandar de C++ y de Linux que se utilizan en los diferentes archivos `.h` y `.cpp`.

```C++
#include <string>
#include <vector>

#include <fstream>
#include <stdexcept>
#include <termios.h>
#include <chrono>
#include <ctime>
#include <sstream>
#include <iomanip>
#include <iostream>
#include <cstdlib>
#include <sys/stat.h>
#include <unistd.h>
#include <pwd.h>
#include <filesystem>
#include <system_error>
#include <algorithm>

```

## ¿Para qué sirven los `#include`?

- `<string>`: `std::string`, cadenas decentes, no arrays C.

- `<vector>`: `std::vector<T>`, arrays dinámicos.

- `<fstream>`: `std::ifstream/ofstream/fstream` para leer/escribir archivos.

- `<stdexcept>`: excepciones estándar (`std::runtime_error`, `std::invalid_argument…`).

- `<termios.h>`: control POSIX de puertos/terminales. Configurar serial: baudrate, paridad, flags.

- `<chrono>`: tiempo moderno C++: steady_clock, milliseconds, sleep_for.

- `<ctime>`: tiempo estilo C: `time_t`, `std::localtime`, `std::strftime-like`.

- `<sstream>`: `std::stringstream` para parsear/armar strings como si fueran streams.

- `<iomanip>`: formateo de streams: `std::setw`, `std::setprecision`, `std::put_time`.

- `<iostream>`: `std::cout`, `std::cin`, `std::cerr`. El trío inevitable.

- `<cstdlib>`: utilidades varias: `std::getenv`, `std::system`, `std::strtol`, `std::rand`.

- `<sys/stat.h>`: stat, permisos y mkdir con modos; funciones del Sistema Operativo.

- `<unistd.h>`: POSIX: read, write, close, usleep, access, isatty.

- `<pwd.h>`: info de usuario: getpwuid para home, etc.

- `<filesystem>`: C++17 paths y archivos: `std::filesystem::path`, `exists`, `create_directories`.

- `<system_error>`: `std::error_code` y `std::system_error` para reportes de errores del Sistema Operativo.

- `<algorithm>`: `std::sort`, `std::find`, `std::transform`, `std::accumulate…` herramientas útiles.

# Manual de instrucciones de la aplicación

##  Uso (manual breve)

### Compilación

Desde el directorio del código (./Codigos):

```bash
make
```

### Captura (modo escritura)

Utilizandose como μC un ESP32 de placa de desarrollo DOIT Devkit v1, se ve el directorio `/dev/ttyACMX` para hacer lectura de la comunicación serial entre el μC y la aplicación (equipo).

Se escribe en la carpeta donde se realizó el make:
```bash
./app -m w -i c -n 10 -s /dev/ttyACMO -d logs -f sensor
```

Donde:
• `-m w`: modo escritura
• `-i c|j|x`: formato de entrada esperado (CSV/JSON/XML “liviano”)
• `-n`: cantidad de lecturas
• `-s`: dispositivo serie
• `-d` y `-f`: directorio y nombre base del archivo

### Lectura/serialización (modo lectura)

```bash
./app -m r -o c -d logs -f sensor # CSV
./app -m r -o j -d logs -f sensor # JSON
./app -m r -o x -d logs -f sensor # XM
```

## Flujo en modo escritura
1. El operador define el formato esperado mediante `-i`.
2. `App` abre/crea el CSV en el directorio indicado y realiza, si es necesario, la cabecera `#h:`.
3. Para cada muestra se envía el byte de *handshake* (`c`, `j` o `x`) y se espera la respuesta del µC.
4. `parseLine` detecta el formato real, normaliza la estructura y devuelve un vector de campos.
5. `File::writeParsed` escapa los valores y los persiste siempre en CSV.
6. Se contabilizan las lecturas correctas y, al finalizar, se informa la tabla de metadatos.


## Flujo en modo lectura
1. `CLI` determina el formato deseado (`-o`).
2. `File` abre el CSV, actualiza los metadatos y expone `getInfoTable()`.
3. En función del formato elegido se reutiliza el CSV original (`toCSV()`), se arma un arreglo JSON (`toJSON()`) o un documento XML (`toXML()`).
4. Cuando no existen encabezados explícitos se generan identificadores genéricos (`c1`, `c2`, ...).


## Serialización y robustez
- Se configuran 115200 bauds, modo 8N1 sin control de flujo hardware.
- El `timeout` de lectura se fijó en 2000 ms para compensar jitter en la transmisión.
- Las líneas incompletas o vacías se descartan sin afectar el contador de lecturas exitosas.
- `File::countRowsOnDisk()` y `readAllRows()` ignoran comentarios y filas vacías para mantener una dimensión realista del dataset.


## Pruebas realizadas
- Compilación con `make` (usar desde `Codigos/`).
- Captura de 20 muestras desde el µC con `./app -m w -i c -n 20 -s /dev/ttyACM0 -d logs -f sensor`.
- Visualización en los tres formatos disponibles: `./app -m r -o c`, `./app -m r -o j`, `./app -m r -o x`.
- Verificación manual de la consistencia del CSV resultante (`logs/sensor.csv`).


# Conclusiones

La aplicación cumple con los objetivos del TP: separa responsabilidades, estandariza formatos heterogéneos y simplifica la interacción con el µC mediante un protocolo simple de handshakes. La arquitectura facilita extender nuevas fuentes/destinos de datos y admitir otros formatos. 


# Referencias consultadas

Documentación de la biblioteca estándar de C++ (containers, streams, <filesystem>).

Páginas de manual POSIX para termios, read/write/close, stat, getpwuid.

Apuntes de cátedra de POO (sección “Guía de Trabajos Prácticos”).




# Anexo

## Esquema de clases

- **App** usa **CLI**, **File** y opcionalmente **SerialPort**

- **File** compone **FileInfo** y utiliza `std::fstream`.

- **CLI** agrupa `Mode` e `IOFormat`; **Types** concentra `AppError` y utilidades.